In [ ]:
%load_ext autoreload

# IMPORT LIBS

In [ ]:
import os
import sys
import json
import numpy as np
import time
from PIL import Image, ImageDraw
from pathlib import Path
import torch, torchvision
import numpy as np
import os, json, cv2, random
from detectron2.utils.visualizer import ColorMode
import glob
import imageio
import cv2
import matplotlib.pyplot as plt
import random
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

ROOT_DIR = 'F:/detectron2/'
assert os.path.exists(ROOT_DIR)
print(torch.__version__, torch.cuda.is_available())

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,default_argument_parser, default_setup, launch
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, COCOPanopticEvaluator, DatasetEvaluators, SemSegEvaluator
from detectron2.data.datasets import register_coco_panoptic_separated

# REGISTER DATASET
- register_coco_panoptic
- register_coco_panoptic_separated

Here, you just need to update the paths to the dataset.

In [ ]:
register_coco_panoptic_separated(name="train",
                                 metadata={},
                                 image_root="F:/detectron2/datasets/bsb_dataset/image_train",
                                 panoptic_root="F:/detectron2/datasets/bsb_dataset/panoptic_train",
                                 sem_seg_root="F:/detectron2/datasets/bsb_dataset/panoptic_stuff_train",
                                 panoptic_json="F:/detectron2/datasets/bsb_dataset/annotations/panoptic_train.json",
                                 instances_json="F:/detectron2/datasets/bsb_dataset/annotations/instance_train.json")

register_coco_panoptic_separated(name="val",
                                 metadata={},
                                 image_root="F:/detectron2/datasets/bsb_dataset/image_val",
                                 panoptic_root="F:/detectron2/datasets/bsb_dataset/panoptic_val",
                                 sem_seg_root="F:/detectron2/datasets/bsb_dataset/panoptic_stuff_val",
                                 panoptic_json="F:/detectron2/datasets/bsb_dataset/annotations/panoptic_val.json",
                                 instances_json="F:/detectron2/datasets/bsb_dataset/annotations/instance_val.json")


register_coco_panoptic_separated(name="test",
                                 metadata={},
                                 image_root="F:/detectron2/datasets/bsb_dataset/image_test",
                                 panoptic_root="F:/detectron2/datasets/bsb_dataset/panoptic_test",
                                 sem_seg_root="F:/detectron2/datasets/bsb_dataset/panoptic_stuff_test",
                                 panoptic_json="F:/detectron2/datasets/bsb_dataset/annotations/panoptic_test.json",
                                 instances_json="F:/detectron2/datasets/bsb_dataset/annotations/instance_test.json")

In [ ]:
city_metadata = MetadataCatalog.get("train_separated")
dataset_dicts = DatasetCatalog.get("train_separated")

city_metadata_val = MetadataCatalog.get('val_separated')
dataset_dicts_val = DatasetCatalog.get('val_separated')

city_metadata_test = MetadataCatalog.get('test_separated')
dataset_dicts_test = DatasetCatalog.get('test_separated')

In [ ]:
MetadataCatalog.get("train_separated").set(stuff_classes=['',
                                                          'street',
                                                          'permeable area',
                                                          'lake'])

MetadataCatalog.get("train_separated").set(thing_classes=['swimming pool',
                                                          'harbor',
                                                          'vehicle',
                                                          'boat',
                                                          'sports court',
                                                          'soccer field',
                                                          'comm. building',
                                                          'comm. building block',
                                                          'res. building',
                                                          'house',
                                                          'small construction'])

MetadataCatalog.get("train_separated").set(stuff_dataset_id_to_contiguous_id={0: 0,
                                                                              1: 1,
                                                                              2: 2,
                                                                              3: 3})

MetadataCatalog.get("train_separated").set(thing_dataset_id_to_contiguous_id={4: 0,
                                                                              5: 1,
                                                                              6: 2,
                                                                              7: 3,
                                                                              8: 4,
                                                                              9: 5,
                                                                              10: 6,
                                                                              11: 7,
                                                                              12: 8,
                                                                              13: 9,
                                                                              14: 10})

MetadataCatalog.get("train_separated").stuff_colors = [(0,0,0), (128,128,128), (0,128,0), (200,0,0)]

#val
MetadataCatalog.get("val_separated").set(stuff_classes=['',
                                                        'street',
                                                        'permeable area',
                                                        'lake'])

MetadataCatalog.get("val_separated").set(thing_classes=['swimming pool',
                                                        'harbor',
                                                        'vehicle',
                                                        'boat',
                                                        'sports court',
                                                        'soccer field',
                                                        'comm. building',
                                                        'comm. building block',
                                                        'res. building',
                                                        'house',
                                                        'small construction'])

MetadataCatalog.get("val_separated").set(stuff_dataset_id_to_contiguous_id={0: 0,
                                                                            1: 1,
                                                                            2: 2,
                                                                            3: 3})

MetadataCatalog.get("val_separated").set(thing_dataset_id_to_contiguous_id={4: 0,
                                                                            5: 1,
                                                                            6: 2,
                                                                            7: 3,
                                                                            8: 4,
                                                                            9: 5,
                                                                            10: 6,
                                                                            11: 7,
                                                                            12: 8,
                                                                            13: 9,
                                                                            14: 10})

MetadataCatalog.get("val_separated").stuff_colors = [(0,0,0), (128,128,128), (0,128,0), (200,0,0)]

#test
MetadataCatalog.get("test_separated").set(stuff_classes=['',
                                                        'street',
                                                        'permeable area',
                                                        'lake'])

MetadataCatalog.get("test_separated").set(thing_classes=['swimming pool',
                                                        'harbor',
                                                        'vehicle',
                                                        'boat',
                                                        'sports court',
                                                        'soccer field',
                                                        'comm. building',
                                                        'comm. building block',
                                                        'res. building',
                                                        'house',
                                                        'small construction'])

MetadataCatalog.get("test_separated").set(stuff_dataset_id_to_contiguous_id={0: 0,
                                                                            1: 1,
                                                                            2: 2,
                                                                            3: 3})

MetadataCatalog.get("test_separated").set(thing_dataset_id_to_contiguous_id={4: 0,
                                                                            5: 1,
                                                                            6: 2,
                                                                            7: 3,
                                                                            8: 4,
                                                                            9: 5,
                                                                            10: 6,
                                                                            11: 7,
                                                                            12: 8,
                                                                            13: 9,
                                                                            14: 10})
MetadataCatalog.get("test_separated").stuff_colors = [(0,0,0), (128,128,128), (0,128,0), (200,0,0)]

# SHOW IMAGES & ANNOTATIONS

## SHOW SINGLE IMAGE WITH ANNOTATION
- This shows an example on how to show a given image from the dataset and its corresponding panoptic annotation.

In [ ]:
#choose image number
img_number = 21

img_path = 'F:/detectron2/datasets/bsb_dataset/image_test/' + str(img_number) + '.tiff'
image = imageio.imread(img_path)
    
img_copy = image.copy()
visualizer = Visualizer(img_copy, metadata=city_metadata_test, scale=1)
vis = visualizer.draw_dataset_dict(dataset_dicts_test[img_number])
    
NUM_ROWS = 1
IMGs_IN_ROW = 2
f, ax = plt.subplots(NUM_ROWS, IMGs_IN_ROW, figsize=(20,20))
    
ax[0].imshow(img_copy/300)
ax[1].imshow(vis.get_image()[:, :, ::-1])

ax[0].set_title('Original image | '+str(img_number))
ax[1].set_title('Panoptic Annotation | '+str(img_number))

print('ID:'+str(img_number+1))
plt.tight_layout()

plt.show()

## SHOW MANY IMAGES WITH ANNOTATIONS
- This shows an example on how to show many images at a time using a for loop.

In [ ]:
for img_number in range(0, 100):
    
    img_path = 'F:/detectron2/datasets/bsb_dataset/image_val/' + str(img_number+1) + '.tiff'
    image = imageio.imread(img_path)
    
    img_copy = image.copy()
    visualizer = Visualizer(img_copy, metadata=city_metadata_val, scale=0.9)
    vis = visualizer.draw_dataset_dict(dataset_dicts_val[img_number])
    
    NUM_ROWS = 1
    IMGs_IN_ROW = 2
    f, ax = plt.subplots(NUM_ROWS, IMGs_IN_ROW, figsize=(20,20))
    
    ax[0].imshow(img_copy/300)
    ax[1].imshow(vis.get_image()[:, :, ::-1])

    ax[0].set_title('Original image | '+str(img_number+1))
    ax[1].set_title('Panoptic Annotation | '+str(img_number+1))

    print('ID:'+str(img_number+1))
    plt.tight_layout()

    plt.show()

# Create a Trainer
- Please read the Detectron2 Trainer documentation if you wish to have more information.

In [ ]:
class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluator_list = []
        evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type

        evaluator_list.append(COCOEvaluator(dataset_name, cfg, True, output_folder))
        evaluator_list.append(COCOPanopticEvaluator(dataset_name, output_folder))
        evaluator_list.append(SemSegEvaluator(dataset_name, distributed=True, output_dir=output_folder))
        
        if len(evaluator_list) == 0:
            raise NotImplementedError(
                "no Evaluator for the dataset {} with the type {}".format(
                    dataset_name, evaluator_type
                )
            )
        elif len(evaluator_list) == 1:
            return evaluator_list[0]
            
        return DatasetEvaluators(evaluator_list)

    @classmethod
    def test_with_TTA(cls, cfg, model):
        logger = logging.getLogger("detectron2.trainer")
        # In the end of training, run an evaluation with TTA
        # Only support some R-CNN models.
        logger.info("Running inference with test-time augmentation ...")
        model = GeneralizedRCNNWithTTA(cfg, model)
        evaluators = [
            cls.build_evaluator(
                cfg, name, output_folder=os.path.join(cfg.OUTPUT_DIR, "inference_TTA")
            )
            for name in cfg.DATASETS.TEST
        ]
        res = cls.test(cfg, model, evaluators)
        res = OrderedDict({k + "_TTA": v for k, v in res.items()})
        return res

# TRAIN

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()

# Choose model
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))

cfg.DATASETS.TRAIN = ("train_separated",)
cfg.DATASETS.TEST = ("test_separated",)

cfg.DATALOADER.NUM_WORKERS = 0

# Number of images per batch
cfg.SOLVER.IMS_PER_BATCH = 1

# Learning Rate
cfg.SOLVER.BASE_LR = 0.00005

# Number of epochs
cfg.SOLVER.MAX_ITER = 1 #1000

cfg.BATCH_SIZE_PER_IMAGE = 512

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 11 # number of classes
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 4

cfg.MODEL.BACKBONE.FREEZE_AT = 4

cfg.TEST.EVAL_PERIOD = 1000 #2500

# Load pre trained weights
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml")
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_best.pth")

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trainer = DefaultTrainer(cfg)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# INFERENCE ON IMAGE

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
import matplotlib.pyplot as plt

img_number = 53

img_path = 'F:/detectron2/datasets/bsb_dataset/image_val/' + str(img_number) + '.tiff'
image = imageio.imread(img_path)

#cfg = get_cfg()
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0
predictor = DefaultPredictor(cfg)
panoptic_seg, segments_info = predictor(image)["panoptic_seg"]
v = Visualizer(image[:, :, ::-1], metadata=city_metadata_test, scale=1)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
plt.figure(figsize=(10,10))

plt.axis('off')

plt.imshow(out.get_image()[:, :, ::-1])

#uncomment below to save image
#plt.savefig('imgs/val_' + str(img_number) + '.png', dpi=400,  bbox_inches='tight', pad_inches=0)
plt.show()